In [17]:
import numpy as np

In [18]:
class LineWorld:
    def __init__(self):
        self.grid = np.arange(13)
        self.states = self.grid.size
        self.actions = 2
        self.state = 6
        self.forbidden = 0
        self.game_over = False
        self.rewards = np.zeros(self.states)
        self.rewards[3] = 1
        
        self.transitions = np.zeros((self.states, self.actions, self.states))
        for s in range(self.states):
            if s > 0:
                self.transitions[s, 0, s-1] = 0.9
                self.transitions[s, 0, s] = 0.1
            if s < self.states - 1:
                self.transitions[s, 1, s+1] = 0.9
                self.transitions[s, 1, s] = 0.1

        for a in range(self.actions):
            self.transitions[12, a, 12] = 1

        for s in range(self.states):
            for a in range(self.actions):
                total = np.sum(self.transitions[s, a])
                if total == 0:
                    self.transitions[s, a] = np.zeros(self.states)
                else:
                    self.transitions[s, a] /= total

    def num_states(self) -> int:
        return self.grid.size

    def num_actions(self) -> int:
        return self.actions

    def num_rewards(self) -> int:
        return self.rewards.size

    def reward(self, i: int) -> float:
        return self.rewards[i]

    def p(self, s: int, a: int, s_p: int) -> float:
        return self.transitions[s, a, s_p]

    def state_id(self) -> int:
        return self.state

    def reset(self):
        self.state = 6
        self.forbidden = 0
        self.game_over = False

    def display(self):
        print(f"State: {self.state}")

    def is_forbidden(self, action: int) -> int:
        return self.forbidden

    def is_game_over(self) -> bool:
        return self.game_over

    def available_actions(self) -> np.ndarray:
        return np.array([0, 1])

    def step(self, action: int):
        if self.game_over:
            raise ValueError("Game is over")
        if self.is_forbidden(action):
            raise ValueError("Forbidden action")
        next_state = np.random.choice(np.arange(self.states), p=self.transitions[self.state, action])
        reward = self.rewards[next_state]
        self.state = next_state
        self.game_over = self.state == 3
        return self.state, reward, self.game_over, {}

    def score(self):
        return self.rewards[self.state]

    def is_terminal(self, state):
        return state == 12

In [19]:
class PolicyIteration:
    def __init__(self, env, gamma=0.9, theta=0.001, max_iter=1000):
        self.env = env
        self.gamma = gamma
        self.theta = theta 
        self.max_iter = max_iter
        self.policy = np.zeros(self.env.num_states(), dtype=int)
        self.value_function = np.zeros(self.env.num_states())

    def policy_iteration(self):
        for _ in range(self.max_iter):
            self.value_function = self.policy_evaluation()

            policy_stable = True
            for s in range(self.env.num_states()):
                old_action = self.policy[s]
                self.policy[s] = self.greedy_policy(s)
                if old_action != self.policy[s]:
                    policy_stable = False

            if policy_stable:
                break

        return self.policy, self.value_function

    def policy_evaluation(self):
        value_function = np.zeros(self.env.num_states())
        for _ in range(self.max_iter):
            delta = 0
            for s in range(self.env.num_states()):
                v = value_function[s]
                value_function[s] = self.expected_value(s)
                delta = max(delta, abs(v - value_function[s]))
            if delta < self.theta:
                break
        self.value_function = value_function
        return value_function

    def expected_value(self, s):
        action = self.policy[s]
        expected_value = 0
        for s_prime in range(self.env.num_states()):
            transition_prob = self.env.p(s, action, s_prime, action)
            reward = self.env.reward(s_prime)
            expected_value += transition_prob * (reward + self.gamma * self.value_function[s_prime])
        return expected_value

    def greedy_policy(self, s):
        actions = self.env.available_actions(s)
        best_action = actions[0]
        best_value = self.expected_value_for_action(s, actions[0])
        for action in actions[1:]:
            value = self.expected_value_for_action(s, action)
            if value > best_value:
                best_action = action
                best_value = value
        return best_action

    def expected_value_for_action(self, s, action):
        expected_value = 0
        for s_prime in range(self.env.num_states()):
            transition_prob = self.env.p(s, action, s_prime, action)
            reward = self.env.reward(s_prime)
            expected_value += transition_prob * (reward + self.gamma * self.value_function[s_prime])
        return expected_value

In [20]:
class MonteCarloES:
    def __init__(self, env, gamma=0.9, episodes=1000):
        self.env = env
        self.gamma = gamma
        self.episodes = episodes
        self.policy = np.random.choice(self.env.available_actions(), size=self.env.num_states())
        self.value_function = np.zeros((self.env.num_states(), self.env.num_actions()))
        self.returns = {(state, action): [] for state in range(self.env.num_states()) for action in range(self.env.num_actions())}

    def generate_episode(self, start_state=None, start_action=None):
        episode = []
        self.env.reset()
        state = start_state if start_state is not None else self.env.state_id()
        action = start_action if start_action is not None else np.random.choice(self.env.available_actions())
        
        steps = 0
        while True:
            self.env.step(action)  # Perform the action
            next_state = self.env.state_id()  # Get the next state
            reward = self.env.reward(next_state)  # Get the reward for the next state
            done = self.env.is_game_over()  # Check if the episode is done
            
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
            action = np.random.choice(self.env.available_actions())
            steps += 1
            if steps > 1000:
                print("Episode too long, stopping")
                break
        
        return episode

    def monte_carlo_es(self):
        for episode_num in range(self.episodes):
            start_state = np.random.choice(self.env.num_states())
            start_action = np.random.choice(self.env.available_actions())
            episode = self.generate_episode(start_state, start_action)
            G = 0
            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G = self.gamma * G + reward
                if (state, action) not in [(x[0], x[1]) for x in episode[:t]]:
                    self.returns[(state, action)].append(G)
                    self.value_function[state, action] = np.mean(self.returns[(state, action)])
                    best_action = np.argmax(self.value_function[state, :])
                    self.policy[state] = best_action

            if episode_num % 100 == 0:
                print(f"Episode {episode_num}/{self.episodes} completed")
        
        # Return the learned policy and value function
        return self.policy, self.value_function

    def find_best_path(self, start_state):
        state = start_state
        path = [state]
        steps = 0
        while True:
            action = self.policy[state]
            self.env.step(action)  # Perform the action
            state = self.env.state_id()  # Get the next state
            path.append(state)
            if self.env.is_game_over():
                break
            steps += 1
            if steps > 1000:
                print("Path too long, stopping")
                break
        return path

In [21]:
class MonteCarloOnPolicy:
    def __init__(self, env, gamma=0.9, epsilon=0.1, episodes=1000):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.episodes = episodes
        self.num_states = self.env.num_states()
        self.num_actions = self.env.num_actions()
        
        # Initialisation de la politique stochastique avec une distribution uniforme
        self.policy = np.ones((self.num_states, self.num_actions)) * (self.epsilon / self.num_actions)
        self.state_action_values = np.zeros((self.num_states, self.num_actions))
        self.returns = {(state, action): [] for state in range(self.num_states) for action in range(self.num_actions)}

    def generate_episode(self):
        self.env.reset()
        state = self.env.state_id()
        episode = []
        done = False
        
        while not done:
            action_probs = self.policy[state]
            action_probs = action_probs / np.sum(action_probs)  # Normaliser les probabilités
            action = np.random.choice(self.num_actions, p=action_probs)
            self.env.step(action)
            next_state = self.env.state_id()
            reward = self.env.reward(next_state)
            done = self.env.is_game_over()
            
            episode.append((state, action, reward))
            state = next_state
        
        return episode

    def monte_carlo_on_policy(self):
        for episode_num in range(self.episodes):
            episode = self.generate_episode()
            G = 0
            visited_state_action_pairs = set()
            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G = self.gamma * G + reward
                if (state, action) not in visited_state_action_pairs:
                    visited_state_action_pairs.add((state, action))
                    self.returns[(state, action)].append(G)
                    self.state_action_values[state][action] = np.mean(self.returns[(state, action)])
                    
                    # Mise à jour de la politique
                    best_action = np.argmax(self.state_action_values[state])
                    for a in range(self.num_actions):
                        if a == best_action:
                            self.policy[state][a] = 1 - self.epsilon + self.epsilon / self.num_actions
                        else:
                            self.policy[state][a] = self.epsilon / self.num_actions
                    # Normaliser la politique pour l'état donné
                    self.policy[state] = self.policy[state] / np.sum(self.policy[state])

            if episode_num % 100 == 0:
                print(f"Episode {episode_num}/{self.episodes} completed")
        
        # Retourner la politique et les valeurs d'état-action
        return self.policy, self.state_action_values

    def find_best_path(self, start_state):
        state = start_state
        path = [state]
        steps = 0
        while True:
            action_probs = self.policy[state]
            action = np.argmax(action_probs)  # Choisir l'action avec la probabilité la plus élevée
            self.env.step(action)
            state = self.env.state_id()
            path.append(state)
            if self.env.is_game_over():
                break
            steps += 1
            if steps > 1000:
                print("Path too long, stopping")
                break
        return path

In [22]:
class MonteCarloOffPolicy:
    def __init__(self, env, gamma=0.9, epsilon=0.1, episodes=1000):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.episodes = episodes
        self.num_states = self.env.num_states()
        self.num_actions = self.env.num_actions()

        # Initialisation de la politique cible (policy) et de la politique comportementale (b_policy)
        self.policy = np.ones((self.num_states, self.num_actions)) * (self.epsilon / self.num_actions)
        self.b_policy = np.ones((self.num_states, self.num_actions)) * (self.epsilon / self.num_actions)
        self.state_action_values = np.zeros((self.num_states, self.num_actions))
        self.C = np.zeros((self.num_states, self.num_actions))  # compteur pour la moyenne des retours

        for state in range(self.num_states):
            best_action = np.random.choice(self.num_actions)
            self.b_policy[state][best_action] += 1 - self.epsilon
    
    def generate_episode(self, policy):
        self.env.reset()
        state = self.env.state_id()
        episode = []
        done = False
        
        while not done:
            action_probs = policy[state]
            action_probs = action_probs / np.sum(action_probs)  # Normaliser les probabilités
            action = np.random.choice(self.num_actions, p=action_probs)
            self.env.step(action)
            next_state = self.env.state_id()
            reward = self.env.reward(next_state)
            done = self.env.is_game_over()
            
            episode.append((state, action, reward))
            state = next_state
        
        return episode

    def monte_carlo_off_policy(self):
        for episode_num in range(self.episodes):
            episode = self.generate_episode(self.b_policy)
            G = 0
            W = 1
            visited_state_action_pairs = set()
            
            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G = self.gamma * G + reward
                
                if (state, action) not in visited_state_action_pairs:
                    visited_state_action_pairs.add((state, action))
                    self.C[state][action] += W
                    self.state_action_values[state][action] += (W / self.C[state][action]) * (G - self.state_action_values[state][action])
                    
                    # Mise à jour de la politique cible
                    best_action = np.argmax(self.state_action_values[state])
                    self.policy[state] = np.zeros(self.num_actions)
                    self.policy[state][best_action] = 1.0

                if action != np.argmax(self.state_action_values[state]):
                    break

                # Mise à jour des poids d'importance
                W = W / (self.b_policy[state][action] + 1e-10)
            
            if episode_num % 100 == 0:
                print(f"Episode {episode_num}/{self.episodes} completed")
        
        # Retourner la politique et les valeurs d'état-action
        return self.policy, self.state_action_values

    def find_best_path(self, start_state):
        state = start_state
        path = [state]
        steps = 0
        while True:
            action_probs = self.policy[state]
            action = np.argmax(action_probs)  # Choisir l'action avec la probabilité la plus élevée
            self.env.step(action)
            state = self.env.state_id()
            path.append(state)
            if self.env.is_game_over():
                break
            steps += 1
            if steps > 1000:
                print("Path too long, stopping")
                break
        return path

In [23]:
# env = LineWorld()
# policy_iteration = PolicyIteration(env)
# policy, value_function = policy_iteration.policy_iteration()
# print("Politique optimale : ", policy)
# print("Fonction de valeur optimale : ", value_function)

In [24]:
env = LineWorld()
monte_carlo_es = MonteCarloES(env, episodes=100)
value, value_function = monte_carlo_es.monte_carlo_es()
print("Politique optimale : ", value)
print("Fonction de valeur optimale : ", value_function)

Episode 0/100 completed
Politique optimale :  [0 0 0 0 0 0 0 0 0 0 0 0 0]
Fonction de valeur optimale :  [[4.10097324e-01 1.26839901e-01]
 [1.70851083e-01 2.34179818e-04]
 [1.31417612e-01 1.61527053e-02]
 [6.43977000e-01 6.73023862e-02]
 [9.34743503e-01 2.54089235e-01]
 [5.09066418e-01 2.09627363e-01]
 [3.10386214e-01 1.00547700e-01]
 [1.82912021e-01 7.92477883e-02]
 [1.31746311e-01 4.19154770e-02]
 [9.59068791e-02 4.07547916e-02]
 [5.81366862e-02 2.63312983e-02]
 [4.83670355e-02 1.52034104e-02]
 [3.06411896e-02 2.00321368e-02]]


In [25]:
env = LineWorld()
monte_carlo_on_policy = MonteCarloOnPolicy(env, episodes=100)
value, value_function = monte_carlo_on_policy.monte_carlo_on_policy()
print("Politique optimale : ", value)
print("Fonction de valeur optimale : ", value_function)

Episode 0/100 completed
Politique optimale :  [[0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]
 [0.95 0.05]]
Fonction de valeur optimale :  [[0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 7.21009950e-01]
 [8.88480000e-01 5.15817721e-01]
 [7.78000472e-01 5.71570060e-01]
 [6.97410000e-01 5.90490000e-01]
 [6.56100000e-01 3.48678440e-01]
 [3.87420489e-01 1.66771817e-01]
 [1.85302019e-01 2.50315550e-02]
 [2.78128389e-02 6.96198609e-04]
 [8.59504456e-04 7.73554010e-04]]


In [26]:
env = LineWorld()
monte_carlo_off_policy = MonteCarloOffPolicy(env, episodes=100)
value, value_function = monte_carlo_off_policy.monte_carlo_off_policy()
print("Politique optimale : ", value)
print("Fonction de valeur optimale : ", value_function)

Episode 0/100 completed
Politique optimale :  [[0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]
 [0.05 0.05]]
Fonction de valeur optimale :  [[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [1.         0.81362427]
 [0.9        0.76846154]
 [0.81       0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]
